<a href="https://colab.research.google.com/github/yyqbb/DATA301-Project/blob/main/DATA301_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install dask

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import urllib.request
import gzip
import shutil
import json
import dask.bag as db
import numpy as np

# Download the compressed file
filename = "renttherunway_final_data.json.gz"
url = "https://datarepo.eng.ucsd.edu/mcauley_group/data/renttherunway/renttherunway_final_data.json.gz"
urllib.request.urlretrieve(url, filename)

# Decompress the file
decompressed_filename = "renttherunway_final_data.json"
with gzip.open(filename, "rb") as f_in:
    with open(decompressed_filename, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

print("File decompressed successfully:", decompressed_filename)

def filter_data_and_update(filename):
    updated_entries = []
    with open(filename, 'r') as f:
        for line in f:
            entry = json.loads(line)
            if "body type" in entry and "rating" in entry and "review_text" in entry and "fit" in entry:
                updated_entries.append({
                    "body type": entry["body type"],
                    "rating": entry["rating"],
                    "review_text": entry["review_text"],
                    "fit": entry["fit"]  # Include the "fit" field
                })

    # Write the updated entries back to the file, each entry on a new line
    with open(filename, 'w') as f:
        for entry in updated_entries:
            f.write(json.dumps(entry) + '\n')

    print("File updated successfully")

# Update the file
filter_data_and_update(decompressed_filename)

# Move the updated file to Google Drive
file_path = '/content/drive/My Drive/processed_data.json'
shutil.copy(decompressed_filename, file_path)
print("Data saved to Google Drive:", file_path)

File decompressed successfully: renttherunway_final_data.json
File updated successfully
Data saved to Google Drive: /content/drive/My Drive/processed_data.json


In [ ]:
import dask.bag as db

# Load the processed data from Google Drive
data_bag = db.read_text(file_path, blocksize="1MB").map(json.loads)

# Display the Dask bag
print(data_bag)
data_bag.take(5) # Print first 5 entries to verify


dask.bag<loads, npartitions=67>


({'body type': 'hourglass',
  'rating': '10',
  'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
  'fit': 'fit'},
 {'body type': 'straight & narrow',
  'rating': '10',
  'review_text': 'I rented this dress for a photo shoot. The theme was "Hollywood Glam and Big Beautiful Hats". The dress was very comfortable and easy to move around in. It is definitely on my list to rent again for another formal event. ',
  'fit': 'fit'},
 {'body type': 'pear',
  'rating': '8',
  'review_text': "I rented this for my company's black tie awards banquet.  I liked that this dress was short but was a little fancier with the sequins. (I generally don't care for long dresses.)  I would describe the color as more rose gold than yellow gold.  I have blonde hair and fair skin, and the color was very flattering.  This is a v